In [ ]:
from ipywidgets import interact
from IPython.display import display
import numpy as np
import dp_stats as dps
%matplotlib inline
import matplotlib.pyplot as plt

## Differentially Private Mean
---

The following tutorial gives one example of how the `dp_mean()` function is called. The data samples are randomly drawn from a Gaussian distribution. The output of the `dp_mean()` function will be compared to a non-differentially private version of the sample mean: $\bar{x}=\frac{1}{n}\sum_{i=1}^{n}x_i$. 

The parameters that can be adjusted are:

- Epsilon
- Delta
- Sample_size

In [ ]:
# This tutorial gives an example of using the dp_mean() function
# The true sample mean and differentially private mean of the data vector will be displayed for comparison

max_sample_size = 100000

# generate a sample data vector
data_ = np.random.normal(loc = 0, scale = 1.0, size = max_sample_size)

# This function will allow the outputs of the means to be interactive
def show_mean(data,Epsilon=1.0, Delta = 0.1, Sample_size = 100):
    
    data = data[:Sample_size]
    
    # restric data vector to be positive and within the range [0, 1]
    data_ = abs(data)
    data_ = data_.clip(min = 0, max = 1.0)

    # find the non-differentially private mean of the generated data
    mean_control = (np.sum(data_) * 1.0) / (Sample_size * 1.0)
    
    # find the differentially private mean of the generated data
    # dp_mean( data_vect, epsilon=1.0, delta=0.1 )
    mean_dp = dps.dp_mean(data_, epsilon = Epsilon, delta = Delta)
    
    # output the control and differentially private mean
    control_txt = 'Non-private Mean: {}'.format(round(mean_control, 4))
    display(control_txt)
    dp_txt = 'Differentially Private Mean: {}'.format(round(float(mean_dp), 4))
    display(dp_txt)
    
show_mean_interact = lambda Epsilon, Delta, Sample_size: show_mean(data_,Epsilon, Delta, Sample_size )

interact(show_mean_interact, Epsilon=(0.01,3,0.01), Delta=(0.01,0.5,0.01), Sample_size=(100,max_sample_size,500));

Each run of the cell generates a new dataset, changing the sample size uses a different amount of the same sample, changing other parameters re-privatizes the same dataset.

It can be noted from the outputs that the differentially private mean will roughly come closer to the actual sample mean when the sample size becomes larger with fixed privacy level, or the privacy level becomes small (Epsilon being large) with fixed sample size.

## Differentially Private Variance
---

The following tutorial gives one example of how the `dp_var()` funciton is called. The data samples are randomly drawn from a Gaussian distribution. The output of the `dp_var()` function will be compared to a non-differentially private version of the sample variance: $\sigma^2=\frac{1}{n}\sum_{i=1}^{n}(x_i-\bar{x})^2$, where $\bar{x}$ is the sample mean. 

The parameters that can be adjusted are:

- Epsilon
- Delta
- Sample_size

In [ ]:

# generate a sample data vector
data_ = np.random.normal(loc = 0, scale = 1.0, size = max_sample_size)

# This function will allow the output of the variances to be interactive
def show_variance(data,Epsilon=1.0, Delta = 0.1, Sample_size = 100):
    data_ = data[:Sample_size]
    
    # restric data vector to be positive and within the range [0, 1]
    data_ = abs(data_)
    data_ = data_.clip(min = 0, max = 1.0)

    # find the non-differentially private mean of the generated data
    mean_control = (np.sum(data_) * 1.0) / (Sample_size * 1.0)
    var_control = 0
    for i in range(Sample_size):
        var_control += (data_[i] - mean_control)**2
    var_control = var_control / (Sample_size * 1.0)
    
    # find the differentially private variance of the generated data
    # dp_var( data_vector,epsilon=1.0,delta=0.1 )
    var_dp = dps.dp_var(data_, epsilon = Epsilon, delta = Delta)
    
    # output the control and differentially private variance
    control_txt = "Non-private Variance: {}".format(round(var_control, 4))
    display(control_txt)
    dp_txt = "Differentially Private Variance: {}".format(round(float(var_dp), 4))
    display(dp_txt)
    
show_variance_interact = lambda Epsilon, Delta, Sample_size: show_variance(data_,Epsilon=1.0, Delta = 0.1, Sample_size = 100)


interact(show_variance_interact, Epsilon=(0.01,3.0,0.01), Delta=(0.01,0.5,0.01), Sample_size=(100,max_sample_size,100));

Each run of the cell generates a new dataset, changing the sample size uses a different amount of the same sample, changing other parameters re-privatizes the same dataset.

It can be noted from the outputs that the differentially private variance will roughly come closer to the actual sample variance when the sample size becomes larger with fixed privacy level, or the privacy level becomes small (Epsilon being large) with fixed sample size.

## Differentially Private PCA
---

The following tutorial gives one example of how the `dp_pca()` funciton is called. The data samples are randomly drawn i.i.d. from a multivariate Gaussian distribution with a pre-defined mean and covariance matrix. The quality (in terms of the captured energy of the covariance matrix in the reduced dimensional subspace) of the output subspace of the differentially private PCA and non-differentially private PCA is shown as a comparison. 

The parameters that can be adjusted are:

- Epsilon
- Sample_size

In [ ]:

# This tutorial gives an example of one way to use the differentially private PCA function
# A non-differentially private version of the PCA process will also be run to generate the likeness of the two


# This function will be used to randomly generate a data matrix from a multivariate Gaussian distribution
def gen_data(Sample_size, k):
    """
    Inputs:
            Sample_size: total number of test samples to return in data matrix
    Outputs:
            trn_data: [trn_size x d]
            A: covariance matrix, [d x d]
    """
    
    import numpy as np

    d = 10                       # features
    n = Sample_size              # number of samples to generate for each class

    # create covariance matrix
    A = np.zeros((d,d))
    for i in range(d):
        if i < k:
            A[i,i] = d - i
        else:
            A[i, i] = 1

    # create mean
    mean = np.zeros(d)

    # generate n samples
    data_ = np.random.multivariate_normal(mean, A, n)    # [nxd]

    return data_, A

# This function will allow the PCA outputs to be interactive
def show_pca_qual(Sample_size, k = 5, Epsilon = 1.0):
    import numpy as np
    import dp_stats as dps
    
    # generate the data matrix
    data_, A = gen_data(Sample_size, k)    # data_: samples are in rows, A: covariance matrix
    
    # go through the non-differentially private PCA routine
    sigma_control = np.dot(data_.T, data_)     # [d x d] = [d x Sample_size] [Sample_size x d]
    U, S, V = np.linalg.svd(sigma_control)
    
    # grab the first k columns
    U_reduc = U[:, :k]
    
    # find the quality of the PCA control
    control_quality = np.trace(np.dot(np.dot(U_reduc.T, A), U_reduc))
    
    
    # go through the differentially private PCA routine
    # dp_pca_sn ( data, epsilon=1.0 )  // samples must be in columns
    sigma_dp = dps.dp_pca_sn(data_.T, epsilon = Epsilon)
    U_dp, S_dp, V_dp = np.linalg.svd(sigma_dp)
    
    # grab the first k columns
    U_dp_reduc = U_dp[:, :k]
    
    # find the quality of the differentially private PCA method
    dp_quality = np.trace(np.dot(np.dot(U_dp_reduc.T, A), U_dp_reduc))
    
    # output the results
    control_txt = "Non-private Quality: {}".format(round(control_quality, 4))
    display(control_txt)
    dp_txt = "Differentially Private Quality: {}".format(round(float(dp_quality), 4))
    display(dp_txt)

interact(show_pca_qual, Sample_size=(50,1000,100), k=(1, 10, 1), Epsilon=(0.01,3.0,0.01));

## Differentially Private SVM Algorithm Tutorial


The following tutorial gives one example of how a differentially private pipeline of PCA and SVM functions are called. The data set used is generated from randomly drawn samples from a mulitivariate Gaussian distribution. A non-differentially private pipeline of PCA and SVM is also utilized. This is used to compare the results (classification accuracy) of the differentially private pipeline.

###### A sample of the pipeline is shown below:

Generate samples --> Perform PCA for dimension reduction --> Perform SVM for classifier training --> Test classifier

The parameters that can be adjusted are:

- Epsilon_pca
- Epsilon_svm

In [ ]:
# This tutorial shows one common pipeline for learning a binary classifier


# This function is used to randomly generate data samples from a multivariate Gaussian distribution.
def gen_data(num_tst_samp, k):
    """
    Inputs:
            num_tst_samp: total number of test samples to return in data matrix
            n: number of samples to generate for each class
    Outputs:
            data: data matrix with samples in rows, [nxd]
            labels: n dimensional vector
    """
    import numpy as np
    import dp_stats as dps

    d = 10    # features
    n = num_tst_samp

    # create covariance matrix
    A = np.zeros((d,d))
    for i in range(d):
        if i < k:
            A[i,i] = d - i
        else:
            A[i, i] = 1

    # create mean for class 1
    mean1 = -1 * np.ones(d)

    # create mean for class 2
    mean2 = np.ones(d)

    # generate n samples for class 1
    cls1_samps = np.random.multivariate_normal(mean1, A, n)    # [nxd]
    # generate n samples for class 2
    cls2_samps = np.random.multivariate_normal(mean2, A, n)    # [nxd]
    return cls1_samps, cls2_samps, A

# This function is used to randomly mix the two class samples and return training and testing data
def sample_selection(data_cls1, data_cls2, trn_size, N):
    """
    Inputs:
            data_cls1: class 1 data, samples are in rows [nxd]
            data_cls2: class 2 data, samples are in rows [nxd]
            trn_size: number of samples to use for training data, integer
            tst_size: number of samples to use for testing data, integer
    Outputs:
            trn_data: [trn_size x d]
            trn_labels:[trn_size]
            tst_data: [tst_size x d]
            tst_labels: [tst_size]
    """
    import numpy as np

    ind = np.random.permutation(N)
    trn_data1 = data_cls1[ind[:trn_size],:]
    trn_data2 = data_cls2[ind[:trn_size],:]
    tst_data1 = data_cls1[ind[trn_size:N],:]
    tst_data2 = data_cls2[ind[trn_size:N],:]

    trn_data = np.concatenate((trn_data1, trn_data2),axis=0)
    trn_labels = np.concatenate( (np.ones(trn_size), -1*np.ones(trn_size)),axis=0 )

    tst_data = np.concatenate((tst_data1, tst_data2),axis=0)
    tst_labels = np.concatenate((np.ones(N-trn_size),-1*np.ones(N-trn_size)),axis=0)

    return trn_data, trn_labels, tst_data, tst_labels

# this function will score the differentially private classifier
def test_dp_clf(data_tst, labels_tst, dp_clf):
    import numpy as np
    
    # loop throught the data and record wrong answers
    n = len(labels_tst)
    tot_err = 0
    
    for i in range(n):
        dp_ans = np.dot(data_tst[i,:], dp_clf)
        if labels_tst[i] < 0:
            if dp_ans >= 0:
                tot_err += 1
        else:
            if dp_ans <= 0:
                tot_err += 1
    accuracy = (n - tot_err) * 1.0 / (n * 1.0)
    return accuracy

# This function is used give the pipeline interactive control.
def svm_pipeline(Epsilon_pca = 0.5, Epsilon_svm = 0.5):
    from sklearn import svm
    import numpy as np
    import dp_stats as dps
    
    # first generate the training and testing data
    cls1, cls2, A = gen_data(num_tst_samp = 2200, k = 4)
    
    trn_data, trn_labels, tst_data, tst_labels = sample_selection(cls1, cls2, trn_size = 1000, N = 1100)
    
    # go through the non-differentially private PCA routine
    sigma_control = np.dot(trn_data.T, trn_data)     # [d x d] = [d x Sample_size] [Sample_size x d]
    U, S, V = np.linalg.svd(sigma_control)
    
    # grab the first k columns
    U_reduc = U[:, :4]      # [d x k]
    
    # project the data onto the k subspace
    trn_data_reduc = np.dot(trn_data, U_reduc)
    
    # go through SVM routine
    clf = svm.SVC(kernel = 'linear', gamma = 0.01, C = 10)
    clf.fit(trn_data_reduc, trn_labels)
    
    # reduce testing data to use to score the control classifier
    tst_data_reduc = np.dot(tst_data, U_reduc)    # [d x k]
    control_score = clf.score(tst_data_reduc, tst_labels)
    #print(control_score)
    
    # go through differentially private pipeline
    # dp_pca_sn ( data, epsilon=1.0 )  // samples must be in columns
    sigma_dp = dps.dp_pca_ag(tst_data.T, epsilon = Epsilon_pca)
    U_dp, S_dp, V_dp = np.linalg.svd(sigma_dp)
    
    # grab the first k columns
    U_dp_reduc = U_dp[:, :4]
    
    # project the data
    dp_trn_data = np.dot(trn_data, U_dp_reduc)
    
    # go through differentially private svm routine
    # dp_svm(data, labels, method='obj', epsilon=0.1, Lambda = 0.01, h = 0.5)
    clf_dp = dps.dp_svm(dp_trn_data, trn_labels, epsilon = Epsilon_svm)
    
    # reduce the testing data
    tst_dp_data = np.dot(tst_data, U_dp_reduc)
    
    # test the differentially private classifier
    dp_score = test_dp_clf(tst_dp_data, tst_labels, clf_dp)
    #print(dp_score)
    
    # output the results
    control_txt = "Non-private Quality: {}".format(round(control_score, 4))
    display(control_txt)
    dp_txt = "Differentially Private Quality: {}".format(round(dp_score, 4))
    display(dp_txt)
    
interact(svm_pipeline, Epsilon_pca=(0.01,1.0,0.05), Epsilon_svm=(0.01,1.0, 0.05));

Note that the pipeline above takes some time, so move the bars slowly to see results

## Differentially Private Histogram
---

The following tutorial gives one example of how the `dp_hist()` function is called. The data samples are randomly drawn from a Gaussian distribution. The output of the `dp_hist()` function will be used to populate a bar graph and show the empirical distribution of the data. In addition, a non-private version of the histogram will be shown as a comparison.

The parameters that can be adjusted are:

- Epsilon
- Sample_size


In [ ]:
# This function will allow the output graphs to be interactive
def plthist(Epsilon=1.0, Sample_size = 100):
    # generate a sample data vector
    data_ = np.random.normal(loc = 0, scale = 1.0, size = Sample_size)

    # get the histogram count and bin edges from the differentially private histogram
    # dp_hist ( data, num_bins=10, epsilon=1.0, delta=0.1, histtype = 'continuous' )
    dp_hist_counts, bin_edges = dps.dp_hist(data = data_, num_bins = 10, epsilon = Epsilon, delta = 0.1)

    dp_hist_counts = dp_hist_counts.clip(min = 0)    # number of items in bin can't be negative
    bin_edges_ = bin_edges[:len(dp_hist_counts)]    # grab only the left edge for each bin

    # get the width for each of the bins
    width_ = []
    for i in range(len(bin_edges) - 1):
        app = bin_edges[i + 1] - bin_edges[i]
        width_.append(app)

    # plot the non-differentially private version of the histogram
    plt.hist(data_, bins=10)
    plt.title('Non-differentially Private Histogram')
    plt.xlabel('Data Distribution')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

    # plot the differentially private version of the histogram
    plt.bar(bin_edges_, dp_hist_counts, width_)
    plt.title('Differentially Private Histogram')
    plt.xlabel('Data Distribution')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

interact(plthist, Epsilon=(0.1,5.0,0.1), Sample_size=(100,10000,100));